<a href="https://colab.research.google.com/github/hi13i/DLNoteClass/blob/master/DLNoteClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最初にやるやつ

In [ ]:
!git clone https://github.com/hi13i/DLNoteClass.git


Cloning into 'DLNoteClass'...
fatal: could not read Username for 'https://github.com': No such device or address


In [2]:
!pip3 install --pre torch torchvision --extra-index-url https://download.pytorch.org/whl/nightly/cpu
!pip3 install torchinfo
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchinfo import summary
from sklearn.metrics import classification_report
import math
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os

import cv2


torch.manual_seed(17)

if not torch.cuda.is_available():
  print('\nGPU が割り当てられていません.')
  print('「編集」>「ノートブックの設定」から「ハードウェア アクセラレータ」を GPU に変更して再実行してください.')
  print('なお「GPU バックエンドに接続できません」と表示される場合にはしばらく経過してから再実行してください.')
else:
  print('\nGPU が利用できます.')
  print('無料版では GPU として Tesla K80 か Tesla T4 が割り当てられます.')
  print('Tesla T4 の方が高性能で学習も速いですが、何度も利用しているユーザには Tesla K80 が割り当てられることが多いようです.\n')
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  print(gpu_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/nightly/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

GPU が利用できます.
無料版では GPU として Tesla K80 か Tesla T4 が割り当てられます.
Tesla T4 の方が高性能で学習も速いですが、何度も利用しているユーザには Tesla K80 が割り当てられることが多いようです.

Wed Jun 14 05:15:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 

In [4]:
!unzip data

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: data/gray/Eight/gray_e110.jpg  
  inflating: data/gray/Eight/gray_e111.jpg  
  inflating: data/gray/Eight/gray_e112.jpg  
  inflating: data/gray/Eight/gray_e114.jpg  
  inflating: data/gray/Eight/gray_e115.jpg  
  inflating: data/gray/Eight/gray_e117.jpg  
 extracting: data/gray/Eight/gray_e12.jpg  
  inflating: data/gray/Eight/gray_e122.jpg  
  inflating: data/gray/Eight/gray_e123.jpg  
  inflating: data/gray/Eight/gray_e124.jpg  
  inflating: data/gray/Eight/gray_e125.jpg  
  inflating: data/gray/Eight/gray_e126.jpg  
  inflating: data/gray/Eight/gray_e127.jpg  
  inflating: data/gray/Eight/gray_e128.jpg  
  inflating: data/gray/Eight/gray_e13.jpg  
  inflating: data/gray/Eight/gray_e130.jpg  
  inflating: data/gray/Eight/gray_e131.jpg  
  inflating: data/gray/Eight/gray_e133.jpg  
 extracting: data/gray/Eight/gray_e134.jpg  
  inflating: data/gray/Eight/gray_e135.jpg  
  inflating: data/gray/Eight/gray_e136.jpg  
  inflating: data/gray/Ei

# つぎに

In [ ]:
test = 'gray/Eight/gray_e1.jpg'
# 対象画像読み込み
img = cv2.imread("gray/Eight/gray_e1.jpg",cv2.IMREAD_COLOR)

# 画像の大きさを取得
height, width, channels = img.shape[:3]
print("width: " + str(width))
print("height: " + str(height))

width: 64
height: 64


In [5]:
data_dir = "gray"
import torch
import torchvision
from torchvision import transforms

# image_sizeやmean, stdはデータに合わせて設定してください。
image_size = 224
mean = (0.5)
std = (0.5)

# trainデータとvalidationデータが入っているディレクトリのパスを指定
train_image_dir = 'data/gray'
val_image_dir = 'data/val'

# trainデータ向けとvalidationデータ向けに、transformを用意します。
# 皆さんのやりたいことに合わせて適宜変更してください。
data_transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(
            image_size, scale=(0.5, 1.0)
        ),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=[-15, 15]),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        transforms.RandomErasing(0.5),
    ]),
    'val': transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

# torchvision.datasets.ImageFolderでデータの入っているディレクトリのパスと
# transformを指定してあげるだけ。
train_dataset = torchvision.datasets.ImageFolder(root=train_image_dir, transform=data_transform['train'])
val_dataset = torchvision.datasets.ImageFolder(root=val_image_dir, transform=data_transform['val'])

# Datasetができたら、dataloaderに渡してあげればOK
batch_size = 32
train_dataLoader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
val_dataLoader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

### ネットワーク: ResNet

In [6]:
class Bottleneck(nn.Module): # ネットワークを定義する際には nn.Module を継承する

  def __init__(self, in_channels, channels, stride=1, downsample=None):
      super(Bottleneck, self).__init__()
      self.conv1 = nn.Conv2d(in_channels, channels, kernel_size=1, bias=False) # nn.Conv2d: 2D 畳み込み層: 出力チャンネル数(channels)だけカーネル(フィルター)が用意される. 各カーネルの個々の値が学習されるパラメータ.
      self.bn1 = nn.BatchNorm2d(channels) # バッチ正規化: 最適化が行いやすくなる
      self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=stride,
                              padding=1, bias=False)
      self.bn2 = nn.BatchNorm2d(channels)
      self.conv3 = nn.Conv2d(channels, channels * 4, kernel_size=1, bias=False)
      self.bn3 = nn.BatchNorm2d(channels * 4)
      self.relu = nn.ReLU(inplace=True) # 活性化関数: これがあるから非線形な関数を表現できる
      self.downsample = downsample
      self.stride = stride

  def forward(self, x): # 学習・推論時に呼ばれる
      shortcut = x

      out = self.conv1(x)
      out = self.bn1(out)
      out = self.relu(out)

      out = self.conv2(out)
      out = self.bn2(out)
      out = self.relu(out)

      out = self.conv3(out)
      out = self.bn3(out)

      if self.downsample is not None:
          shortcut = self.downsample(x)

      out += shortcut
      out = self.relu(out)

      return out

In [7]:
class ResNet(nn.Module):

  def __init__(self, block, layers, num_classes=1000, input_channels=3):
      self.in_channels = 64
      super(ResNet, self).__init__()
      self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3,
                              bias=False)
      self.bn1 = nn.BatchNorm2d(64)
      self.relu = nn.ReLU(inplace=True)
      self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
      self.layer1 = self._make_layer(block, 64, layers[0])
      self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
      self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
      self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
      self.avgpool = nn.AdaptiveMaxPool2d(output_size=(1,1)) # Max プーリング (nn.AdaptiveMaxPool2d: 指定した出力サイズになるようにカーネルサイズが自動で設定される)

      self.dropout = nn.Dropout2d(p=0.5,inplace=True) # ドロップアウト: 過学習防止 (推論時には入力がそのまま出力される)

      self.fc = nn.Linear(512 * 4, num_classes) # 全結合層: 「出力チャンネル数(num_classes)」=「正解ラベル数」になるよう設定

      # パラメータの初期化
      for m in self.modules():
          if isinstance(m, nn.Conv2d):
              init.kaiming_normal_(m.weight)
          elif isinstance(m, nn.BatchNorm2d):
              m.weight.data.fill_(1)
              m.bias.data.zero_()

  def _make_layer(self, block, channels, blocks, stride=1):
      downsample = None
      if stride != 1 or self.in_channels != channels * 4:
        downsample = nn.Sequential(
            nn.Conv2d(self.in_channels, channels * 4,
                      kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(channels * 4),
        )

      layers = []
      layers.append(block(self.in_channels, channels, stride, downsample))
      self.in_channels = channels * 4
      for i in range(1, blocks):
          layers.append(block(self.in_channels, channels))

      return nn.Sequential(*layers)

  def forward(self, x): # 学習・推論時に呼ばれる

      x = self.conv1(x)
      x = self.bn1(x)
      x = self.relu(x)
      x = self.maxpool(x)

      x = self.layer1(x)
      x = self.layer2(x)
      x = self.layer3(x)
      x = self.layer4(x)

      x = self.avgpool(x)
      x = self.dropout(x)
      x = x.view(x.size(0), -1) # ベクトル化
      x = self.fc(x)

      return x

In [8]:
def get_ResNet(model_type: str, num_classes: int):
  layers_types = {'ResNet50': [3, 4, 6, 3], 'ResNet101': [3, 4, 23, 3], 'ResNet152': [3, 8, 36, 3]}
  assert model_type in layers_types
  model = ResNet(Bottleneck, layers_types[model_type], num_classes)
  return model

### 学習

In [10]:
class AverageMeter(object):
  def __init__(self):
      self.reset()

  def reset(self):
      self.val = 0
      self.avg = 0
      self.sum = 0
      self.count = 0

  def update(self, val, n=1):
      self.val = val
      self.sum += val * n
      self.count += n
      self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(dim=0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [11]:
class Trainer():
  def __init__(self, train_loader, model_type, optimizer_type, num_classes, batch_size, criterion, start_epoch, epochs, lr, use_scheduler, print_freq):

    self.train_loader = train_loader
    self.model_type = model_type
    self.num_classes = num_classes
    self.batch_size = batch_size
    self.criterion = criterion
    self.start_epoch = start_epoch
    self.epochs = epochs
    self.lr = lr
    self.use_scheduler = use_scheduler
    self.print_freq = print_freq

    self.model = get_ResNet(self.model_type, self.num_classes)

    if optimizer_type == 'sgd':
      self.optimizer = optim.SGD(self.model.parameters(),lr=self.lr)
    elif optimizer_type == 'adam':
      self.optimizer = optim.Adam(self.model.parameters(),lr=self.lr,betas=(0.9,0.99),eps=1e-09)

  def get_model(self):
    return self.model

  def save_model(self, path):
    torch.save(self.model.state_dict(), path)

  def load_model(self, path):
    self.model.load_state_dict(torch.load(path))
    return self.model

  def train(self):

    self.model.cuda()
    self.criterion.cuda()

    if self.use_scheduler:
      lr_scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=[20,40,60,80,100,120,140,160,180,200], gamma=0.5)

    losses_record = []
    top1_record = []

    for epoch in tqdm(range(self.start_epoch, self.epochs)):

      epoch_time = AverageMeter()
      losses = AverageMeter()
      top1 = AverageMeter()

      end = time.time()

      for i, (input, target) in enumerate(tqdm(self.train_loader, leave=False)):

        target = target.cuda()
        input_var = input.cuda()
        target_var = target

        output = self.model(input_var)
        loss = self.criterion(output, target_var)

        self.optimizer.zero_grad() # 勾配を初期化
        loss.backward() # 勾配を計算
        self.optimizer.step() # ネットワークパラメータを更新(最適化)

        output = output.float()
        loss = loss.float()

        # 正解率の計算と損失の記録
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        losses_record.append(losses.val)

      epoch_time.update(time.time() - end)

      if epoch % self.print_freq == 0:
        top1_record.append(top1.val)
        print(f'Epoch: {epoch}\t'
              f'Loss: {losses.val:.4f} ({losses.avg:.4f})\t'
              f'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
              f'time: {(epoch_time.val / 60):.2f} min')
      if self.use_scheduler:
        lr_scheduler.step()

    # 損失関数の loss をグラフにプロット
    x = list(range(len(losses_record)))
    y = losses_record
    plt.plot(x, y)
    plt.title(f'{self.criterion}')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.show()

### 評価

In [12]:
def validate(val_loader, model, criterion):

    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    pred_list = []
    target_list =[]

    model.cuda()
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input_var = input.cuda()
            target_var = target.cuda()

            output = model(input_var)
            loss = criterion(output, target_var)

            output = output.float()
            loss = loss.float()

            prec1 = accuracy(output.data, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))

            batch_time.update(time.time() - end)

            _, pred = output.topk(1, 1, True, True)
            pred_list.extend(pred.squeeze().cpu())
            target_list.extend(target_var.cpu())

            end = time.time()

    print(f'Test\t  Prec@1: {top1.avg:.3f} (Err: {100-top1.avg:.3f} )\n')

    return top1.avg, pred_list, target_list

### 実行

In [23]:
# goole drive にマウント (必要に応じてコメントアウト)
# from google.colab import drive
# drive.mount('/content/drive')
# main_dir = '/content/drive/MyDrive/深層学習を用いたCG・画像処理'
# if os.path.isdir('/content/drive/MyDrive/'):
#   os.makedirs(main_dir, exist_ok=True)

In [24]:
# パラメータなどを設定
model_type = 'ResNet50' # ResNetの層数を指定: 'ResNet50' or 'ResNet101' or 'ResNet152'
optimizer_type = 'sgd' # 最適化手法: 'sgd' or 'adam'
num_classes = 5 # データセット中に含まれる画像のラベルの総数を指定: 10 or 100
batch_size = 256 # バッチサイズ: 大きいほど学習が速く進むが、メモリも消費する
criterion = nn.CrossEntropyLoss() # 損失関数: この値が小さくなるように学習(パラメータの更新)を行う
start_epoch = 0
epochs =5 # エポック数
lr = 1e-2 # 学習率 (1e-2 は 0.01 のこと. e は10の冪乗を表す)
use_scheduler=False # スケジューラの利用: スケジューラを使うとエポックごとに学習率を変更できる
print_freq = 1
normalize = False # バッチ単位で画像の正規化の有無

In [25]:
# データセットのダウンロード
# train_loader, test_loader, classes = get_dataset(batch_size, num_classes, normalize, show=True)

In [26]:
# 設定を変更した場合には再実行すること
trainer = Trainer(train_dataLoader, model_type, optimizer_type, num_classes, batch_size, criterion, start_epoch, epochs, lr, use_scheduler, print_freq)
# モデルを可視化
summary(model=trainer.get_model(), input_size=(batch_size, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [256, 5]                  --
├─Conv2d: 1-1                            [256, 64, 16, 16]         9,408
├─BatchNorm2d: 1-2                       [256, 64, 16, 16]         128
├─ReLU: 1-3                              [256, 64, 16, 16]         --
├─MaxPool2d: 1-4                         [256, 64, 8, 8]           --
├─Sequential: 1-5                        [256, 256, 8, 8]          --
│    └─Bottleneck: 2-1                   [256, 256, 8, 8]          --
│    │    └─Conv2d: 3-1                  [256, 64, 8, 8]           4,096
│    │    └─BatchNorm2d: 3-2             [256, 64, 8, 8]           128
│    │    └─ReLU: 3-3                    [256, 64, 8, 8]           --
│    │    └─Conv2d: 3-4                  [256, 64, 8, 8]           36,864
│    │    └─BatchNorm2d: 3-5             [256, 64, 8, 8]           128
│    │    └─ReLU: 3-6                    [256, 64, 8, 8]           --
│ 

In [ ]:
# 学習
trainer.train()

# ネットワークのパラメータを保存 (google drive にマウントすること)
# trainer.save_model(f'{main_dir}/model_weight_{model_type}_{epochs}ep_lr={lr}.pth')

# 評価
_, pred_list, target_list = validate(val_dataLoader, trainer.get_model(), criterion)
# 適合率(precision)、再現率(recall)、F1スコア、正解率(accuracy)、マクロ平均、マイクロ平均を計算して表示
print(classification_report(target_list, pred_list))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 0	Loss: 1.8802 (20.9368)	Prec@1 25.000 (26.229)	time: 0.68 min


  0%|          | 0/110 [00:00<?, ?it/s]

In [ ]:
# google drive に保存済みのモデルを読み込んで利用する場合
# model = get_ResNet(model_type, num_classes)
# model.load_state_dict(torch.load(f'{main_dir}/model_weight_{model_type}_{epochs}ep_lr={lr}.pth'))
# _, pred_list, target_list  = validate(test_loader, model, criterion)
# print(classification_report(target_list, pred_list, target_names=classes))